In [ ]:
'''
2017年10月12日
   完成:
        对test.mp4视频进行训练
        eye_img=UI.drew_face_eye(frame,minNeighbors=10,scaleFactor=1.1,minSize=(20,20))
'''

In [1]:
import tensorflow as tf
import numpy as np
from cumt_eye import Cumt_itraker
import matplotlib.pyplot as plt
import os
from scipy.misc import imread
from scipy.misc import imresize
import UI
import cv2
sess=tf.InteractiveSession()
cumt_data=np.load(r'D:\Proj_DL\Code\Proj_EyeTraker\cumt_data.pkl')


X=tf.placeholder(shape=[None,32,128,3],dtype=tf.float32)
Y=tf.placeholder(shape=[None,10],dtype=tf.float32)
DROPOUT=tf.placeholder(dtype=tf.float32)
LR=tf.placeholder(dtype=tf.float32)
BN_FLAG=tf.placeholder(dtype=tf.bool)

with tf.name_scope('model'):
    model=Cumt_itraker(image=X,bn_flag=BN_FLAG,dropout_rate=DROPOUT)
    y_score=model.score

with tf.name_scope('LOSS'):
    LOSS=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_score,labels=Y))
    tf.summary.scalar('loss',LOSS)
with tf.name_scope('TRAIN'):
    TRAIN=tf.train.AdamOptimizer(LR).minimize(LOSS)
with tf.name_scope('ACCURACY'):
    acc_count=tf.equal(tf.arg_max(y_score,1),tf.arg_max(Y,1))
    ACCURACY=tf.reduce_mean(tf.cast(acc_count,tf.float32))
    tf.summary.scalar('acc',ACCURACY)

writer_tr=tf.summary.FileWriter('./mylog/test')
writer_te=tf.summary.FileWriter('./mylog/val')
merge=tf.summary.merge_all()
ex_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)



Cumt_itraker Init Ready!


In [ ]:
'''
在视频中插入判别网络
'''

In [2]:
#从保存的检查点钟复原变量
saver=tf.train.Saver()
saver.restore(sess,r'D:\Proj_DL\Code\Proj_EyeTraker\Proj_iTraker\CUMT_iTraker\model_save\v2model_99.ckpt-2550')
pic_mean=[103.939, 116.779,123.68 ]

INFO:tensorflow:Restoring parameters from D:\Proj_DL\Code\Proj_EyeTraker\Proj_iTraker\CUMT_iTraker\model_save\v2model_99.ckpt-2550


In [3]:
prob=tf.nn.softmax(y_score,name='prob')
y_guess=tf.arg_max(prob,1)+1

#预测方向到九宫格方向,
pre2block={2:3,3:2,4:1,
           5:6,6:5,7:4,
           8:9,9:8,10:7}

In [4]:
cap = cv2.VideoCapture(r'demo.flv')
#cap = cv2.VideoCapture(r'C:\Users\Administrator\Downloads\Video\ee.flv')
fps=cap.get(cv2.CAP_PROP_FPS)
fsize=(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
      int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),)
# save video
video_saver=cv2.VideoWriter('demo13.flv',cv2.VideoWriter_fourcc('F','L','V','1'),fps,fsize)
while (True):
    ret,frame = cap.read()
    if ret==True:
        #ori 为原始图片--bgr,网络训练是用rgb训练的
        ori_img=frame.copy()
        #截取眼部图片
        eye_img=UI.drew_face_eye(frame,minNeighbors=10,scaleFactor=1.1,minSize=(20,20))
        
        if eye_img is None:
            #没有检测到眼睛，跳到下一帧
            cv2.imshow('frame',ori_img)
            video_saver.write(frame)
            cv2.waitKey(30)
            continue
        #e1=imresize(eye_img,(32,128,3)).astype('float32')-mean_pic
        e1=imresize(eye_img,(32,128,3)).astype('float32')-[123.68, 116.779, 103.939]
        #eye_img=(eye_img-[123.68, 116.779, 103.939])[:,:,::-1]
        #检测到眼睛，送入网络
        #x_=imresize(eye_img,(32,128,3)).reshape((1,32,128,3))
        x_=e1.reshape((1,32,128,3))
        pre_asw=sess.run(y_guess,feed_dict={X:x_,DROPOUT:1.,BN_FLAG:False})[0]
        #给图片划线，划框
        frame=UI.drawline(frame,line_w=1)
        #if pre_asw == 1:pre_asw=6
        #print(pre_asw,pre2block[pre_asw])
        if pre_asw == 1:
            #闭眼的情况
#             UI.drawblock(frame,1)
#             UI.drawblock(frame,2)
#             UI.drawblock(frame,3)
            pass
        else:
            #有检测结果
            
            frame=UI.drawblock(frame,pre2block[pre_asw])
            #cv2.waitKey()
        #cv2.imshow('frame',x_.reshape((32,128,3)))
        cv2.imshow('frame',frame)
        video_saver.write(frame)
        if cv2.waitKey(30)& 0xff ==ord('q'):
            print('quite')
            break
    else:
        print('video end')
        break
cap.release()
video_saver.release()
cv2.destroyAllWindows()

video end
